In [132]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv


In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4') 
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_dat

In [4]:
data = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv')
#data.sample(5)
test_data = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv')
test_labels = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv')
merged_data = pd.merge(test_data, test_labels, on='id')

# Filter rows where toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate' are not -1
filtered_test_data = merged_data[
    (merged_data['toxic'] != -1) &
    (merged_data['severe_toxic'] != -1) &
    (merged_data['obscene'] != -1) &
    (merged_data['threat'] != -1) &
    (merged_data['insult'] != -1) &
    (merged_data['identity_hate'] != -1)
]
filtered_test_data.to_csv('/kaggle/working/cleaned_test_data.csv', index=False)
filtered_test_data.sample(200)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
142535,ee43b2a3b27138b4,== WP:HOUND == \n\n You are hounding SamJ. Th...,0,0,0,0,0,0
88489,93987491916b0490,===Flames archived for posterity=== \n ==== Ch...,0,0,0,0,0,0
80766,86cc9ec23bc6542d,== A pic: inertial circles == \n\n I don't kn...,0,0,0,0,0,0
74449,7c0fed13f39e2536,Hi. I do believe that this is harassment and f...,0,0,0,0,0,0
19439,209d9f429ab3a275,== A belated welcome! == \n\n ]]Here's wishing...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153072,ffddd5898e4687bb,""" \n\n == Citing the First Amendment == \n\n T...",0,0,0,0,0,0
11909,13f4cb060bbfe14c,::' No to warm climate here. But mutabilis loo...,0,0,0,0,0,0
24734,29349b25185d287a,=== Percentage of Shi'a in Turkey === \n\n Shi...,0,0,0,0,0,0
149228,f96f552cd6b939ce,"""*Comment - The lede and Product overview sect...",0,0,0,0,0,0


In [5]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [6]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation and numbers
    text = text.strip()  # Remove leading and trailing whitespaces
    tokens = text.split()  # Tokenize the text by splitting on whitespace
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize the tokens
    cleaned_text = ' '.join(tokens)  # Join tokens back into a single string
    return cleaned_text

In [ ]:
data['cleaned_comment'] = data['comment_text'].apply(clean_text)
data['is_toxic'] = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
filtered_test_data['cleaned_comment'] = filtered_test_data['comment_text'].apply(clean_text)
filtered_test_data['is_toxic'] = filtered_test_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)


/tmp/ipykernel_33/3316149774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_data['cleaned_comment'] = filtered_test_data['comment_text'].apply(clean_text)
/tmp/ipykernel_33/3316149774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_data['is_toxic'] = filtered_test_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)


In [7]:
data['is_toxic'].unique()


array([0, 1])

In [8]:
filtered_test_data['is_toxic'].unique()

array([0, 1])

In [9]:
data.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_comment,is_toxic
5058,0d698e07c775e149,"""\n\n Whole Ariticle Needs to br Rewritten \n\...",0,0,0,0,0,0,whole ariticle need br rewritten believe whole...,0
132492,c4e3142842260982,"""\n\nWhat page? No page was put up as an artic...",1,0,0,0,0,0,page page put article per wiki guideline nothi...,1
141179,f363f548c6a0850e,Why is page in half? ==\n\nOn my screen I see ...,0,0,0,0,0,0,page half screen see large space history part ...,0
13027,228324eb05a022d5,"Since consensus was to keep, and I still felt ...",0,0,0,0,0,0,since consensus keep still felt strongly lack ...,0
84529,e21d7c6e831bb0a3,Thank you; I appreciate how you have quickly a...,0,0,0,0,0,0,thank appreciate quickly acknowledged correcte...,0


In [10]:

filtered_test_data.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_comment,is_toxic
69541,73c2e730843f2a19,""" \n\n Stop harrassing me. """,0,0,0,0,0,0,stop harrassing,0
20095,21ad16e269fc026a,:See your talk page.,0,0,0,0,0,0,see talk page,0
87504,91fad40545aeea09,""" \n\n == re:""""Gulosten"""" == \n\n Perhaps. Tha...",0,0,0,0,0,0,regulosten perhaps would nice going dyk first ...,0
36764,3d0aa2875dcc9d61,== Hello from == \n\n Hi there! I'm running ...,0,0,0,0,0,0,hello hi im running training session editathon...,0
147059,f5d62a380bbe5fb7,")|here]], and [[Wikipedia:Articles for deletio...",0,0,0,0,0,0,wikipediaarticles deletioninterdenominational ...,0


In [8]:
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(data['cleaned_comment'])
y = data['is_toxic']
X_test = vectorizer.transform(filtered_test_data['cleaned_comment'])
y_test= filtered_test_data['is_toxic']

In [19]:
# Count occurrences of each distinct value in y_test
label_counts = y_test.value_counts()

# Print the counts
print("Label Distribution in Test Data:")
print(label_counts)

Label Distribution in Test Data:
is_toxic
0    57735
1     6243
Name: count, dtype: int64


In [9]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X,y)


In [10]:
nb = MultinomialNB()
nb.fit(X_train_resampled,y_train_resampled)

MultinomialNB()

In [12]:
y_pred = nb.predict(X_test)

In [13]:
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
y_prob = nb.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
ks_statistic, _ = ks_2samp(y_prob[y_test == 1], y_prob[y_test == 0])

In [14]:
print("auc",auc)
print(f"KS Statistic: {ks_statistic:.2f}")

auc 0.9448434710719429
KS Statistic: 0.75


In [15]:
print(classification_report(filtered_test_data['is_toxic'], y_pred))

              precision    recall  f1-score   support

           0       0.99      0.84      0.91     57735
           1       0.38      0.91      0.54      6243

    accuracy                           0.85     63978
   macro avg       0.68      0.87      0.72     63978
weighted avg       0.93      0.85      0.87     63978



In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from scipy.stats import ks_2samp

# Define the parameter grid with different alpha values
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Numeric values for alpha
}

# Initialize MultinomialNB classifier
nb = MultinomialNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, cv=5, scoring='f1')

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Print accuracy for all alpha values from cv_results_
print("Accuracy for all alpha values:")
for i in range(len(grid_search.cv_results_['params'])):
    alpha = grid_search.cv_results_['params'][i]['alpha']
    mean_score = grid_search.cv_results_['mean_test_score'][i]
    print(f"Alpha: {alpha}, F1-score: {mean_score:.4f}")

# Print the best parameters and the best score
print("\nBest parameters found: ", grid_search.best_params_)
print("Best F1 score found: ", grid_search.best_score_)

# Predict with the best model
best_nb = grid_search.best_estimator_
y_pred_best = best_nb.predict(X_test)

# Evaluate the best model
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))

# Calculate and print accuracy for the best model
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"\nAccuracy of the best model: {accuracy_best:.4f}")

# Print ROC AUC and KS Statistic for the best model
y_prob_best = best_nb.predict_proba(X_test)[:, 1]
auc_best = roc_auc_score(y_test, y_prob_best)  # Corrected here
ks_statistic_best, _ = ks_2samp(y_prob_best[y_test == 1], y_prob_best[y_test == 0])
print("Best Model ROC AUC:", auc_best)
print(f"Best Model KS Statistic: {ks_statistic_best:.2f}")


Accuracy for all alpha values:
Alpha: 0.1, F1-score: 0.8969
Alpha: 0.5, F1-score: 0.8971
Alpha: 1.0, F1-score: 0.8974
Alpha: 1.5, F1-score: 0.8973
Alpha: 2.0, F1-score: 0.8976

Best parameters found:  {'alpha': 2.0}
Best F1 score found:  0.897550473938769

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.84      0.91     57735
           1       0.38      0.91      0.53      6243

    accuracy                           0.84     63978
   macro avg       0.68      0.87      0.72     63978
weighted avg       0.93      0.84      0.87     63978


Accuracy of the best model: 0.8443
Best Model ROC AUC: 0.9460374686072581
Best Model KS Statistic: 0.75
